In [1]:
from IPython.display import HTML
from image_functions import *

Two issues remain to be corrected. The first one was already presented in section *2.1.4 Limitations of the Angle-Based Algorithm.* It concerned the behavior of the angular algorithm for spiral turns that the vertical line does not intersect. The problem was that the angular algorithm lands the radius vector onto the $x$-axis.

In [2]:
load_image('fig135')

Here, we need to make an important clarification about which radius vectors are being landed onto the $x$-axis. Since we have already added the $\text{ISSCDD}$-check, we will see that it deactivates $\text{AB-Alg}$ in some cases but not in others. In the two images below, we show the first initial radius vector and the final radius vector at the $5$-th intersection point.

In [5]:
display(load_images([('fig136', 'left'), ('fig137', 'right')]))

As can be seen, the derivatives have the same sign (positive slope) at the beginning and at the end of the iterative process, and $\text{ISSCDD}$ gives a "green light" for $\text{AB-Alg}$. However, the situation is different for the intersection points on the positive side of the ordinate axis.

In [6]:
display(load_images([('fig138', 'left'), ('fig139', 'right')]))

In this case, on the fourth iteration ($m=3$), the derivative changes its sign, and $\text{ISSCDD}$ nullifies $\text{AB-Alg}$. These behaviors of the algorithm are due to the position of $\text{PMD}$. In the upper case, the "descending" radius vector passes $\text{PMD}$, causing the derivative to change its sign. In the previous case, $\text{PMD}$ is positioned above the abscissa axis (i.e., after it in terms of time $t$), which is why the derivative does not change its sign.